Julia Pawluś, Kamil Sarzyniak, Piotr Stawarski 

In [1]:
#pip install -U pyFTS
import pyFTS

In [15]:
from pyFTS.data import Bitcoin

In [16]:
df = Bitcoin.get_dataframe()

In [17]:
df.head()

,Date,Open,High,Low,Close,Volume,AVG
0,2010-07-16,0.04951,0.04951,0.04951,0.04951,0,0.049510
1,2010-07-17,0.04951,0.08585,0.05941,0.08584,5,0.070152
2,2010-07-18,0.08584,0.09307,0.07723,0.08080,49,0.084235
3,2010-07-19,0.08080,0.08181,0.07426,0.07474,20,0.077902
4,2010-07-20,0.07474,0.07921,0.06634,0.07921,42,0.074875
